In [ ]:
import numpy as np
from matplotlib import pyplot as plt
np.set_printoptions(threshold=np.nan, linewidth=np.nan)
%matplotlib inline

In [ ]:
import pickle
with open('df.pickle', mode='rb') as h:
    df = pickle.load(h)
df_label = df[df['split'] == 'test']['label'].reset_index(drop=True)

In [ ]:
from os import listdir
from os.path import join
import re
models = listdir('../caffemodels/')
iters = [int(re.findall('\d+', x)[0]) for x in models]
iters, models = zip(*sorted(zip(iters, models), key=lambda x: x[0]))
models = [join('../caffemodels/', x) for x in models]
mean_pixel = np.load('../ilsvrc_2012_mean.npy').mean(axis=(1, 2))

In [ ]:
import h5py
with h5py.File('../test.h5', mode='r') as h:
    X = h['data'].value

In [ ]:
with open('hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
name_id = hex_data['name_id']
ihl = map(set, hex_data['id_hierarchical_labels'])  # ihl for id_hierarchical_labels
ifh = map(lambda x: max(x, key=len), ihl)  # ifh for id_full_hierarchy

In [ ]:
import caffe
caffe.set_mode_cpu()
threahold = 0.01
def test_model(caffemodel):
    """
    Type 0: Is the full hierarchy
    Type 1: Covers the full hierarchy
    Type 2: Is any partial hierarchy
    Type 3: Covers any partial hierarchy
    Implication: 1->2, 1->3, 2->4, 3->4
    """
    counter = np.array([0, 0, 0, 0], dtype=np.uint32)
    net = caffe.Classifier(model_file='deploy.prototxt', pretrained_file=caffemodel,
                           mean=mean_pixel, channel_swap=(2, 1, 0),
                           raw_scale=1, image_dims=(227, 227))
    for i in range(0, len(X)):
        y = net.predict([X[i]], oversample=False)
        y /= y.max()
        y = y > threahold
        y = np.nonzero(y)[1]  # np.array
        lid = name_id[df_label[i]]  # lid for label_id
#         print('y={}, hieharchy={}'.format(tuple(y), ihl[lid]))
        if tuple(y) == ifh[lid]:
            counter += 1
#             print('type 0')
        elif set(ifh[lid]) in set(y):
            counter[[1, 3]] += 1
#             print('type 1')
        elif tuple(y) in ihl[lid]:
            counter[[2, 3]] += 1
#             print('type 2')
        elif any(set(x) in set(y) for x in ihl[lid]):
            counter[3] += 1
#             print('type 3')
    return counter

In [ ]:
accuracy = zip(map(test_model, models))

In [ ]:
import pickle
with open('baseline.pickle', mode='wb') as h:
    pickle.dump(accuracy, h)

In [ ]:
fig = plt.figure(figsize=(8, 5))
ax = fix.add_subplot(111)
ax.plot(accuracy[0], iters)
ax.grid()